![Private Eyes are Watching You](art/private-eyes.jpg)

# Fetching Summary Data 
## Introducing the Ontology2 Edition of Dbpedia

In our [last episode](http://ontology2.com/notebooks/local/DBpedia_Schema_Queries.html),  I did a number of queries against the DBpedia Ontology to map out the information available.  In that notebook,  I gave myself the restriction that I would only do queries against a copy of the DBpedia
Ontology that is stored with the notebook.

Because the Ontology contains roughly 740 types and 2700 properties (more than 250 for Person alone)
this turned out to be a serious limitation -- unless we know how much information is available for these properties,  I can't know which ones are important,  and thus make a visualization that makes sense.

Gastrodon is capable of querying the [DBpedia Public SPARQL endpoint](http://dbpedia.org/sparql), but the  DBpedia Endpoint has some limitations,  particularly,  it returns at most 10,000 results for a query.  Complex queries can also time out.  Certainly I could write a series of smaller queries to compute statistics,  but then I face a balancing act between too many small queries (which will take a long time to run) and queries that get too large (and sometimes time out.)

Fortunately I have a product in the AWS Marketplace,  the [Ontology2 Edition of DBpedia 2016-04](https://aws.amazon.com/marketplace/pp/B01HMUNH4Q/) which is a private SPARQL endpoint already loaded with data from DBpedia.  By starting this product,  and waiting about an hour for it to initialize,  I can run as many SPARQL queries as I like of arbitrary complexity,  and shut it down when I'm through.

In this notebook,  I use this private SPARQL endpoint to count the prevalence of types,  properties,  and datatypes.  I use SPARQL Construct to save this information into an RDF graph that I'll later be able to combine with the DBpedia Ontology RDF graph to better explore the schema.


I start with the usual preliminaries,  importing Python modules and prefix definitions

In [30]:
%load_ext autotime
import sys
from os.path import expanduser
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
import pandas as pd
import json
pd.options.display.width=120
pd.options.display.max_colwidth=100

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.5 ms


In [2]:
prefixes=inline("""
    @prefix dbo: <http://dbpedia.org/ontology/> .
    @prefix summary: <http://rdf.ontology2.com/summary/> .
""").graph

time: 8 ms


It wouldn't be safe for me to check database connection information into Git,  so I store it in a file
in my home directory named `~/.dbpedia/config.json`, which looks like

```
{
    "url":"http://130.21.14.234:8890/sparql-auth",
    "user":"dba",
    "passwd":"vKUcW1eSVkruDOtT",
    "base_uri":"http://dbpedia.org/resource/"
}
```

(Note that that is not my real IP address and passwd.  If you want to reproduce this,  put in the IP address and password for your own server and save it to `~/.dbpedia/config.json`

In [3]:
connection_data=json.load(open(expanduser("~/.dbpedia/config.json")))
connection_data["prefixes"]=prefixes

time: 4 ms


In [4]:
endpoint=RemoteEndpoint(**connection_data)

time: 12.5 ms


## Counting Properties and Classes

### Finding the right graphs

The [Ontology2 Edition of DBpedia 2016-04](https://aws.amazon.com/marketplace/pp/B01HMUNH4Q/) is
divided into a number of different named graphs,  one for each dataset described [here](http://wiki.dbpedia.org/downloads-2016-04#datasets).

It's important to pay attention to this for two reasons.

One of them is that facts can appear in the output of a SPARQL query more than once than if the query covers multiple graphs and if facts are repeated in those graphs.  This can throw off the accuracy of our counts.

The other is that some queries seem to take a long time to run if they are run over all graphs;  particularly this affects queries that involve filtering over a prefix in the predicate field (ex.)

```
FILTER(STRSTARTS(STR(?p)),"http://dbpedia.org/ontology/")
```

Considering both of these factors,  it is wise to know which graphs the facts we want are stored in,  thus I start exploring:

In [5]:
endpoint.select("""
    select ?g (COUNT(*) AS ?cnt) {
       GRAPH ?g { ?a <http://dbpedia.org/ontology/Person/height> ?b } .
    } GROUP BY ?g
""")

,cnt
g,
http://downloads.dbpedia.org/2016-04/core-i18n/en/citedFacts_en.ttl.bz2,5502
http://downloads.dbpedia.org/2016-04/core-i18n/en/specific_mappingbased_properties_en.ttl.bz2,148105


time: 2.07 s


Thus I find one motherload of properties right away:  I save this in a variable so I can use it later.

In [6]:
pgraph=URIRef("http://downloads.dbpedia.org/2016-04/core-i18n/en/specific_mappingbased_properties_en.ttl.bz2")

time: 999 µs


Looking up types,  I find a number of graphs and choose the transitive types:

In [8]:
endpoint.select("""
    select ?g (COUNT(*) AS ?cnt) {
       GRAPH ?g { ?a a dbo:Person } .
    } GROUP BY ?g
""")

,cnt
g,
http://downloads.dbpedia.org/2016-04/core-i18n/en/instance_types_transitive_en.ttl.bz2,1014819
http://downloads.dbpedia.org/2016-04/core-i18n/en/instance_types_en.ttl.bz2,502997
http://downloads.dbpedia.org/2016-04/core-i18n/en/instance_types_sdtyped_dbo_en.ttl.bz2,212295
http://downloads.dbpedia.org/2016-04/core-i18n/en/instance_types_lhd_dbo_en.ttl.bz2,834547


time: 332 ms


In [9]:
tgraph=URIRef("http://downloads.dbpedia.org/2016-04/core-i18n/en/instance_types_transitive_en.ttl.bz2")

time: 1 ms


### Counting Classes

It is now straightforward to pull up a list of types (classes),  noting that these are not mutually exclusive.  (You can be a `dbo:Actor` and a `dbo:Politician`)

In [10]:
endpoint.select("""
   SELECT ?type (COUNT(*) AS ?cnt) {
       GRAPH ?_tgraph { ?s a ?type . }
       FILTER(STRSTARTS(STR(?type),"http://dbpedia.org/ontology/"))
    } GROUP BY ?type
""")

,cnt
type,
dbo:WinterSportPlayer,22373
dbo:Project,11
dbo:PopulatedPlace,505557
dbo:Actor,1969
dbo:Document,23799
dbo:Genre,1229
dbo:Group,33681
dbo:Politician,19569
dbo:Station,2300


time: 1min 39s


I can store these facts in an RDF graph (instead of a Pandas DataFrame) by using a `CONSTRUCT` query (instead of a `SELECT` query).  To capture the results of a `GROUP BY` query,  however,  I have to use a subquery -- this is because SPARQL requires that I only use variables in the `CONSTRUCT` clause,  thus I have to evaluate expressions (such as `COUNT(*)`) somewhere else.

The resulting query is straightforward,  even if it looks a little awkward with all the braces:  roughly I cut and pasted the above SELECT query into a `CONSTRUCT` query that defines the facts that will be emitted.

In [11]:
t_counts=endpoint.construct("""
   CONSTRUCT {
      ?type summary:count ?cnt .
   } WHERE { 
       {
            SELECT ?type (COUNT(*) AS ?cnt) {
                GRAPH ?_tgraph { ?s a ?type . }
                FILTER(STRSTARTS(STR(?type),"http://dbpedia.org/ontology/"))
            } GROUP BY ?type
       } 
   }
""")

time: 1min 40s


I can count the facts in this resulting graph (same as the number of rows in the `SELECT` query)

In [31]:
len(t_counts)

108

time: 3 ms


And here is a sample fact:

In [40]:
next(t_counts.__iter__())

(rdflib.term.URIRef('http://dbpedia.org/ontology/Book'),
 rdflib.term.URIRef('http://rdf.ontology2.com/summary/count'),
 rdflib.term.Literal('22', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))

time: 4 ms


Note that in the DBpedia Ontology there are a number of other facts about `dbo:Book`,  so if add the above fact to my copy of the DBpedia Ontology,  SPARQL queries will be able to pick up the count together with all the other facts.

## Counting "Specific Properties"

If I count properties in the "specific mappingbased properties" graph,  I find that these are
all properties that have the Class name baked in

In [12]:
endpoint.select("""
   SELECT ?p (COUNT(*) AS ?cnt) {
       GRAPH ?_pgraph { ?s ?p ?o . }
    } GROUP BY ?p
""")

,cnt
p,
http://dbpedia.org/ontology/Canal/originalMaximumBoatLength,5
http://dbpedia.org/ontology/Engine/length,16
http://dbpedia.org/ontology/Engine/powerOutput,193
http://dbpedia.org/ontology/Planet/averageSpeed,650
http://dbpedia.org/ontology/Planet/density,123
http://dbpedia.org/ontology/Infrastructure/length,23124
http://dbpedia.org/ontology/Lake/volume,1401
http://dbpedia.org/ontology/Person/weight,66144
http://dbpedia.org/ontology/PopulatedPlace/area,41718


time: 418 ms


In [13]:
sp_count=endpoint.construct("""
    CONSTRUCT {
      ?p summary:count ?cnt .
    } WHERE { {
        SELECT ?p (COUNT(*) AS ?cnt) {
           GRAPH ?_pgraph { ?s ?p ?o . }
        } GROUP BY ?p
   } }
""")

time: 416 ms


## Other Ontology properties

That begs the question of in which graphs other properties are stored.  Searching for `dbo:birthDate` I find the location of ordinary Literal properties.  (Which could be a date,  a number or a string)

In [14]:
endpoint.select("""
    select ?g (COUNT(*) AS ?cnt) {
       GRAPH ?g { ?a dbo:birthDate ?b } .
    } GROUP BY ?g
""")

,cnt
g,
http://downloads.dbpedia.org/2016-04/core-i18n/en/mappingbased_literals_en.ttl.bz2,819371
http://downloads.dbpedia.org/2016-04/core-i18n/en/persondata_en.ttl.bz2,730541
http://downloads.dbpedia.org/2016-04/core-i18n/en/citedFacts_en.ttl.bz2,6658


time: 275 ms


A search for `dbo:child` turns up object properties (which point to a URI reference)

In [15]:
endpoint.select("""
    select ?g (COUNT(*) AS ?cnt) {
       GRAPH ?g { ?a dbo:child ?b } .
    } GROUP BY ?g
""")

,cnt
g,
http://downloads.dbpedia.org/2016-04/core-i18n/en/mappingbased_objects_en.ttl.bz2,14456
http://downloads.dbpedia.org/2016-04/core-i18n/en/mappingbased_objects_disjoint_range_en.ttl.bz2,112
http://downloads.dbpedia.org/2016-04/core-i18n/en/citedFacts_en.ttl.bz2,91
http://downloads.dbpedia.org/2016-04/core-i18n/en/mappingbased_objects_uncleaned_en.ttl.bz2,14568


time: 247 ms


In [16]:
lgraph=URIRef("http://downloads.dbpedia.org/2016-04/core-i18n/en/mappingbased_literals_en.ttl.bz2")
ograph=URIRef("http://downloads.dbpedia.org/2016-04/core-i18n/en/mappingbased_objects_en.ttl.bz2")

time: 1 ms


### Counting All Properties

By taking a `UNION` I can count the "specific", object, and literal properties.  The DataFrame looks OK,  so I decide to save these counts into a graph.

In [17]:
endpoint.select("""
   SELECT ?p (COUNT(*) AS ?cnt) {
       { 
           GRAPH ?_pgraph { 
               ?s ?p ?o .      
           }
       } UNION { 
           GRAPH ?_ograph {
               ?s ?p ?o .
           }
       } UNION {
           GRAPH ?_lgraph {
               ?s ?p ?o .
           }
       }
    } GROUP BY ?p
""")

,cnt
p,
dbo:film,20
dbo:headteacher,1
dbo:poleDriverCountry,84
dbo:numberOfClassrooms,2
dbo:conservationStatus,48437
dbo:dateOfBurial,12
dbo:established,2463
dbo:firstPublicationYear,6390
dbo:isniId,50


time: 3.16 s


In [18]:
p_counts=endpoint.construct("""
    CONSTRUCT {
       ?p summary:count ?cnt .
    } WHERE {
        {
            SELECT ?p (COUNT(*) AS ?cnt) {
               { 
                   GRAPH ?_pgraph { 
                       ?s ?p ?o .      
                   }
               } UNION { 
                   GRAPH ?_ograph {
                       ?s ?p ?o .
                   }
               } UNION {
                   GRAPH ?_lgraph {
                       ?s ?p ?o .
                   }
               }
            } GROUP BY ?p
        }
    }
""")

time: 3.09 s


In [19]:
len(p_counts)

1438

time: 2.99 ms


## Counting datatypes

In a RDF,  a Class is a kind of type which represents a "Thing" in the world.  Datatypes,  on the other hand,  are types that represent literal values.  The most famous types in RDF come from the [XML Schema Datatypes](http://www.w3.org/TR/xmlschema-2/) and represent things such as integers,  dates,  and strings.

RDF also allows us to define custom datatypes,  which are specified with URIs,  like most things in RDF.

A `GROUP BY` query reveals the prevalence of various datatypes,  which I then dump to a graph.

There still are some big questions to research such as "does the same property turn up with different units?"  For instance,  it is very possible that a length could be represented in kilometers,  centimeters,  feet,  or furlongs.  You won't get the right answer,  however,  if you try to add multiple lengths in different units that are all represented as floats.  Thus it may be necessary at some point to build a bridge to a package like [numericalunits](https://pypi.python.org/pypi/numericalunits) or alternately build something that canonicalizes them.

In [20]:
endpoint.select("""
   SELECT ?datatype (COUNT(*) AS ?cnt) {
       { 
           GRAPH ?_pgraph { 
               ?s ?p ?o .      
           }
       } UNION {
           GRAPH ?_lgraph {
               ?s ?p ?o .
           }
       }
       BIND(DATATYPE(?o) AS ?datatype)
    } GROUP BY ?datatype
""")

,cnt
datatype,
http://dbpedia.org/datatype/kilometre,36045
http://dbpedia.org/datatype/kelvin,559
http://dbpedia.org/datatype/millimetre,59730
http://dbpedia.org/datatype/centimetre,148105
http://dbpedia.org/datatype/metre,387
http://dbpedia.org/datatype/litre,11
http://dbpedia.org/datatype/newtonMetre,67
xsd:nonNegativeInteger,824348
xsd:string,3571269


time: 1min 36s


In [21]:
dt_counts=endpoint.construct("""
    CONSTRUCT {
       ?datatype summary:count ?cnt .
    } WHERE {
       SELECT ?datatype (COUNT(*) AS ?cnt) {
           { 
               GRAPH ?_pgraph { 
                   ?s ?p ?o .      
               }
           } UNION {
               GRAPH ?_lgraph {
                   ?s ?p ?o .
               }
           }
           BIND(DATATYPE(?o) AS ?datatype)
        } GROUP BY ?datatype
    }
""")

time: 1min 35s


## Writing to disk

RDFlib overloads the '+' operator so that we can easily merge the type,  property and datatype counts into one (modestly sized) graph.

In [25]:
all_counts = t_counts + p_counts + dt_counts

time: 182 ms


I add a few prefix declarations for (human) readability,  then write the data to disk in Turtle format.  I was tempted to write it to a relative path which would put this file in its final destination.  (Underneath the `local` notebook directory,  where it could be found by notebooks) but decided against it,  since I don't want to take the chance of me (or you) trashing the project by mistake.  Instead I'll have to copy the file into place later.

In [28]:
all_counts.bind("datatype","http://dbpedia.org/datatype/")
all_counts.bind("dbo","http://dbpedia.org/ontology/")
all_counts.bind("summary","http://rdf.ontology2.com/summary/")
all_counts.serialize("/data/schema_counts.ttl",format='ttl',encoding='utf-8')

time: 507 ms


## Bonus File: Human Dimensions

While I had my copy of DBpedia running,  I thought I'd gather a data set that would be worth making visualizations of.  Quite a lot of data exists in DBpedia concerning people's body dimensions,  so I decided to run a query and save the data for future use.

In [22]:
dimensions=endpoint.select("""
select ?p ?height ?weight {
    GRAPH ?_pgraph {
        ?p <http://dbpedia.org/ontology/Person/weight> ?weight .
        ?p <http://dbpedia.org/ontology/Person/height> ?height .
    }
}
""")

time: 51 s


In [23]:
dimensions

,p,height,weight
0,<Alexander_Hug_(rugby_union)>,188.00,91.000000
1,<Anderson_Silva>,187.96,83.916000
2,<Andrew_Gee>,183.00,102.000000
3,<Bernard_Ackah>,185.42,90.720000
4,<Billy_Brandt>,177.80,74.844000
5,<Bob_Beamon>,191.00,70.000000
6,<Caleb_Moore>,177.80,72.576000
7,<Charles_Hamelin>,175.00,71.000000
8,<Charmaine_Sinclair>,172.72,57.152639
9,<Christina_Von_Eerie>,162.56,55.792800


time: 25 ms


The data looks a bit messy.  Most noticeably,  I see quite a few facts which,  instead of pointing to DBpedia concepts,  point to synthetic URLs (such as `<Ron_Clarke__2>`) which are supposed to represent 'topics' such the time that a particular employee worked for a particular employer.  (See [this notebook for some discussion of the phenomenon](http://ontology2.com/notebooks/remote/Querying%20DBpedia.html)).

Filtering these out will not be hard,  as these synthetic URLs all contain two consecutive underscores.

I also think it's suspicious that a few people have a height of `0.0`,  which might be in the underlying data,  or might be because Gastrodon is not properly handling a missing data value.

It would be certainly possible to serialize these results into an RDF graph,  but instead I write them into a CSV for simplicity.

In [24]:
dimensions.to_csv("/data/people_weight.csv.gz",compression="gzip",encoding="utf-8")

time: 504 ms


## Conclusion

To continue the analysis I began [here](http://ontology2.com/notebooks/local/DBpedia_Schema_Queries.html),  I needed a count of how often various classes,  properties,  and datatypes were used in DBpedia.  API limits could make getting this data from the public SPARQL endpoint challenging,  so I decided to run queries against my own
private SPARQL endpoint powered by the [Ontology2 Edition of DBpedia](https://aws.amazon.com/marketplace/pp/B01HMUNH4Q/).

After setting up connection information,  connecting to this private endpoint turned out to be as simple as connecting to a public endpoint and I was efficiently able to get the data I needed into an RDF graph,  ready to merge with the DBpedia Ontology graph to make a more meaningful analysis of the data in DBpedia towards the goal of producing interesting and attractive visualizations.